In [1]:
pip install git+https://github.com/openai/whisper.git openai

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-z7ynpgju
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-z7ynpgju
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803583 sha256=1c0df83f67540fbe346b88f4f7d2c7a5b0d94ac2b5b2f19844be85ac7cf560a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-cmgx9me_/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The program then needs to be initialized. Ensure you place in your audio file name, select Whisper Model to use, and your OpenAI API Key (Don't worry. Google Colab doesn't save your apikey if you are in Playground Mode.)



Now, we will be setting up the transcribe() and gpt_process() functions.

Last, we will run the code. This will output 2 files into the session storage. The Transcript.txt as well as a Summary.txt

In [ ]:
import torch
import whisper
import tqdm
import sys

audio = "1.mp3"  # Убедитесь, что аудиофайл загружен в нужное место!
model = "base"  # Доступные опции: 'tiny', 'base', 'small', 'medium', 'large'


def transcribe(audio, model_type):
    class _CustomProgressBar(tqdm.tqdm):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self._current = self.n

        def update(self, n):
            super().update(n)
            self._current += n

            print("Audio Transcribe Progress: " + str(self._current) + "/" + str(self.total))

    devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = whisper.load_model(model_type, device=devices)
    transcribe_module = sys.modules['whisper.transcribe']
    transcribe_module.tqdm.tqdm = _CustomProgressBar

    print("Beginning Transcribing Process...")
    result = model.transcribe(audio, verbose=None, fp16=False)
    transcribed = result["text"]
    with open("Transcript.txt", "w", encoding='utf-8') as text_file:
        text_file.write(transcribed)
        print("Saved Transcript to Transcript.txt")
    return transcribed

# Транскрибация аудио
text = transcribe(audio, model)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Beginning Transcribing Process...
Audio Transcribe Progress: 2500/3430
Audio Transcribe Progress: 5000/3430
Audio Transcribe Progress: 5930/3430
Audio Transcribe Progress: 6860/3430
Saved Transcript to Transcript.txt


In [ ]:
import torch
import whisper
import tqdm
import sys
from transformers import pipeline

# Параметры
audio = "2.mp3"  # Убедитесь, что аудиофайл загружен в нужное место!
model_type = "base"  # Доступные опции: 'tiny', 'base', 'small', 'medium', 'large'

# Функция транскрибации
def transcribe(audio, model_type):
    class _CustomProgressBar(tqdm.tqdm):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self._current = self.n

        def update(self, n):
            super().update(n)
            self._current += n
            print("Audio Transcribe Progress: " + str(self._current) + "/" + str(self.total))

    devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = whisper.load_model(model_type, device=devices)
    transcribe_module = sys.modules['whisper.transcribe']
    transcribe_module.tqdm.tqdm = _CustomProgressBar

    print("Beginning Transcribing Process...")
    result = model.transcribe(audio, verbose=None, fp16=False)
    transcribed = result["text"]
    with open("Transcript.txt", "w", encoding='utf-8') as text_file:
        text_file.write(transcribed)
        print("Saved Transcript to Transcript.txt")
    return transcribed

# Анализ текста с помощью Hugging Face
def analyze_text(text):
    print("Starting Text Analysis...")

    # Анализ эмоций
    emotion_classifier = pipeline("sentiment-analysis")
    emotions = emotion_classifier(text)

    # Классификация текста (примерно по теме новостей)
    classifier = pipeline("zero-shot-classification")
    categories = classifier(text, candidate_labels=["Complaint", "Review", "Question", "Request", "Feedback"])

    # Суммаризация текста
    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)

    # Сохранение результата
    with open("Analysis.txt", "w", encoding='utf-8') as file:
        file.write("Original Text:\n" + text + "\n\n")
        file.write("Emotion Analysis:\n" + str(emotions) + "\n\n")
        file.write("Text Classification:\n" + str(categories) + "\n\n")
        file.write("Text Summary:\n" + summary[0]['summary_text'] + "\n")
        print("Saved Analysis to Analysis.txt")

# Транскрибация и анализ текста
transcribed_text = transcribe(audio, model_type)
analyze_text(transcribed_text)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Beginning Transcribing Process...
Audio Transcribe Progress: 2616/6691
Audio Transcribe Progress: 5232/6691
Audio Transcribe Progress: 7848/6691
Audio Transcribe Progress: 10464/6691
Audio Transcribe Progress: 13012/6691
Audio Transcribe Progress: 15560/6691
Audio Transcribe Progress: 18108/6691
Audio Transcribe Progress: 20656/6691


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Audio Transcribe Progress: 22183/6691
Audio Transcribe Progress: 23710/6691
Audio Transcribe Progress: 25237/6691
Audio Transcribe Progress: 26764/6691
Saved Transcript to Transcript.txt
Starting Text Analysis...


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Saved Analysis to Analysis.txt


In [ ]:
import torch
import whisper
import tqdm
import sys
from transformers import pipeline
from sentence_transformers import SentenceTransformer

# Параметры
audio = "1.mp3"  # Убедитесь, что аудиофайл загружен в нужное место!
model_type = "base"  # Доступные опции: 'tiny', 'base', 'small', 'medium', 'large'

# Функция транскрибации
def transcribe(audio, model_type):
    class _CustomProgressBar(tqdm.tqdm):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self._current = self.n

        def update(self, n):
            super().update(n)
            self._current += n
            print("Audio Transcribe Progress: " + str(self._current) + "/" + str(self.total))

    devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = whisper.load_model(model_type, device=devices)
    transcribe_module = sys.modules['whisper.transcribe']
    transcribe_module.tqdm.tqdm = _CustomProgressBar

    print("Beginning Transcribing Process...")
    result = model.transcribe(audio, verbose=None, fp16=False)
    transcribed = result["text"]
    with open("Transcript.txt", "w", encoding='utf-8') as text_file:
        text_file.write(transcribed)
        print("Saved Transcript to Transcript.txt")
    return transcribed

# Анализ текста с помощью Hugging Face
def analyze_text(text):
    print("Starting Text Analysis...")

    # Анализ эмоций
    emotion_classifier = pipeline("sentiment-analysis")
    emotions = emotion_classifier(text)

    # Анализ тональности речи
    tone_classifier = pipeline("text-classification", model="facebook/bart-large-mnli")
    tone_analysis = tone_classifier(text)

    # Суммаризация текста
    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)

    # Извлечение ключевых слов и тем
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    keywords = model.encode([text])

    # Сохранение результата
    with open("Analysis.txt", "w", encoding='utf-8') as file:
        file.write("Original Text:\n" + text + "\n\n")
        file.write("Emotion Analysis:\n" + str(emotions) + "\n\n")
        file.write("Tone Analysis:\n" + str(tone_analysis) + "\n\n")
        file.write("Text Summary:\n" + summary[0]['summary_text'] + "\n\n")
        file.write("Key Themes and Keywords:\n" + str(keywords) + "\n")
        print("Saved Analysis to Analysis.txt")

# Транскрибация и анализ текста
transcribed_text = transcribe(audio, model_type)
analyze_text(transcribed_text)
